Install Libraries

In [34]:
!pip install streamlit sentence_transformers pymupdf docx2txt pandas spacy matplotlib
!pip -m spacy download en_core_web_sm


Usage:   
  pip3 <command> [options]

no such option: -m


In [35]:
import fitz, docx2txt, spacy, pandas as pd
from sentence_transformers import SentenceTransformer
print("All libraries installed correctly")

All libraries installed correctly


In [36]:
import fitz
import docx2txt
import re
from google.colab import files

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
!ls /content/drive/MyDrive


 1627984789883.gdoc
 1627984789883.pdf
'20-20 & 50-50 Series (Reg Students)'
'Colab Notebooks'
'Copy of Data Journal: Journal Entry Templates.gdoc'
 DSC_0170.JPG
'Getting started.pdf'
'HD472877_EquityP&L_01-Feb-2021_07-Feb-2021.xls'
'My resume- India - Google Docs-converted.gdoc'
'new doc 2024-08-04 17.46.34.pdf'
'new doc 2024-08-04 17.47.08 (1).pdf'
'new doc 2024-08-04 17.47.08.pdf'
'new doc 2024-08-04 17.47.38.pdf'
'new doc 2024-08-04 17.47.59.pdf'
'new doc 2024-08-04 17.48.19.pdf'
'resume (1).gdoc'
 resume.gdoc
 resume_juanjosecarin.pdf
 resume.pdf
 Statistics
'Untitled spreadsheet.gsheet'
 XXXXXXXXXXX1263-28-11-2022to27-02-2023.pdf


In [39]:
!ls /content/drive/MyDrive/'Colab Notebooks'/

 AI_Resume.Screener.ipynb	     resume_juanjosecarin.pdf
 App.py				    'Text Generation.ipynb'
'Copy of AI_Resume.Screener.ipynb'   the_first.ipynb
'Copy of App.py'		     Untitled0.ipynb
'Kallu Sujeev Reddy.pdf'


Extraction Function

In [40]:
def extract_text(file_path):
  text = "" # Initialize text here

  if file_path.endswith(".pdf"):
    with fitz.open(file_path) as pdf :
      for page in pdf:
        text += page.get_text("text")

  elif file_path.endswith(".docx"):
    text = docx2txt.process(file_path)

  else :
    print("File not supported !!")

  # Basic Cleaning
  text = re.sub(r'\s+', ' ',text).strip()
  return text

In [41]:
file_path = "/content/drive/MyDrive/Colab Notebooks/resume_juanjosecarin.pdf"
text = extract_text(file_path)
print(text[:1000])

1 of 2 Juan Jose Carin Data Scientist Mountain View, CA 94041 650-336-4590 | juanjose.carin@gmail.com linkedin.com/in/juanjosecarin | juanjocarin.github.io Professional Profile Passionate about data analysis and experiments, mainly focused on user behavior, experience, and engagement, with a solid background in data science and statistics, and extensive experience using data insights to drive business growth. Education 2016 University of California, Berkeley Master of Information and Data Science GPA: 3.93 Relevant courses: • Machine Learning • Machine Learning at Scale • Storing and Retrieving Data • Field Experiments • Applied Regression and Time Series Analysis • Exploring and Analyzing Data • Data Visualization and Communication • Research Design and Applications for Data Analysis 2014 Universidad Politécnica de Madrid M.S. in Statistical and Computational Information Processing GPA: 3.69 Relevant courses: • Data Mining • Multivariate Analysis • Time Series • Neural Networks and St

Text Extraction

In [42]:
import os
folder_path = "/content/drive/MyDrive/Colab Notebooks/" # Corrected to the folder path

for file_name in os.listdir(folder_path):
  if file_name.endswith(".pdf") or file_name.endswith("docx"):
    file_path = os.path.join(folder_path, file_name) # Corrected filename to file_name
    extracted = extract_text(file_path)
    print(f"File : {file_name}")
    print(f"Extracted text sample :/n{extracted[:500]}")

File : resume_juanjosecarin.pdf
Extracted text sample :/n1 of 2 Juan Jose Carin Data Scientist Mountain View, CA 94041 650-336-4590 | juanjose.carin@gmail.com linkedin.com/in/juanjosecarin | juanjocarin.github.io Professional Profile Passionate about data analysis and experiments, mainly focused on user behavior, experience, and engagement, with a solid background in data science and statistics, and extensive experience using data insights to drive business growth. Education 2016 University of California, Berkeley Master of Information and Data Scienc
File : Kallu Sujeev Reddy.pdf
Extracted text sample :/nKALLU SUJEEV REDDY Hyderabad, IN | sujeevreddy93@gmail.com | 9100780854 | https://www.linkedin.com/in/sujeevreddykallu/ PROFESSIONAL SUMMARY Innovative Data Scientist with experience building LLM-driven workflows and autonomous agents for Finance and HR operations. Skilled in designing NLP pipelines, anomaly detection systems, and AI- powered chatbots for document summarization and re

Extract Structured Data

In [43]:
nlp = spacy.load("en_core_web_sm")

In [44]:
def extract_structured_data(text):
    data = {
        "Name": None,
        "Email": None,
        "Phone": None,
        "Education": [],
        "Experience": [],
        "Skills": []
    }

## ----- Extract EMAIL -----

    email_match = re.search(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', text)
    if email_match:
        data["Email"] = email_match.group(0)

## ----- Extract PHONE ----

    phone_match = re.search(r'(\+?\d{1,3}[\s-]?)?\(?\d{3,5}\)?[\s.-]?\d{3,5}[\s.-]?\d{3,5}', text)
    if phone_match:
        data["Phone"] = phone_match.group(0)

## ---- Extract SKILLS ----

    skill_keywords = [
        "python", "excel", "sql", "machine learning", "deep learning", "communication", "power bi",
        "tableau", "data analysis", "r", "java", "c++", "tensorflow", "pytorch", "nlp"
    ]
    found_skills = [skill for skill in skill_keywords if skill.lower() in text.lower()]
    data["Skills"] = found_skills

## ---- Extract NAME -----

  ## to extract name from top section of resume

    top_section = "\n".join(text.split("\n")[:10]).strip()   # takes only first 10 lines

 ## to remove lines that contain emails and phone numbers

    top_section = re.sub(r'[\w\.-]+@[\w\.-]+', '', top_section)
    top_section = re.sub(r'(\+?\d{1,3}[\s-]?)?\(?\d{2,4}\)?[\s.-]?\d{3,5}[\s.-]?\d{3,5}', '', top_section)

 ## Run spaCY NER

    doc = nlp(top_section)

## Lookin for PERSON entities

    name_candidates = [ent.text.strip() for ent in doc.ents if ent.label_ == "PERSON"] # Corrected ent.label to ent.label_

    if name_candidates:
      data["Name"] = name_candidates[0] # Assign the found name to data["Name"]

    # """for line in top_section.split("\n"):
    #    line = line.strip()
    #    if line and re.match(r'^[A-Za-z\s\.-]{3,}$', line) and len(line.split()) <= 4:
    #        return line """ # Commented out and indented the block


## ----- Extract EDUCATION -----

  ## Extracting Education based on "Education heading". Captures everything until next major heading

    text_lower = text.lower()

    next_sections = ['experience', 'project', 'projects', 'skill', 'skills', 'award', 'awards', 'certification', 'certifications']

  ## Finding the start of Education section

    edu_match = re.search(r'\beducation\b', text_lower)
    if not edu_match:
     return data # Return data if no education section found

    edu_start = edu_match.start() # Corrected variable name


  ## Initialising the end of Education section

    edu_end = len(text)

  ## Finding the closest next section after Education

    for sec in next_sections:
        sec_match = re.search(r'\b' + re.escape(sec) + r'\b', text_lower[edu_start + 1:])
        if sec_match:
            potential_end = edu_start + 1 + sec_match.start()
            if potential_end < edu_end:
                edu_end = potential_end

  ## Extracting and cleaning the section
    education_text = text[edu_start:edu_end].strip()

    education_text = re.sub(r'\n{2,}', '\n', education_text)
    data["Education"] = education_text # Assign the extracted education text to data["Education"]


## ----- Extract EXPERIENCE -----

    text_lower = text.lower()

    # Define possible next section headers
    next_sections = ['education', 'project', 'projects', 'skill', 'skills', 'award', 'awards', 'certification', 'certifications']

    # Find where the 'experience' section starts
    exp_match = re.search(r'\bexperience\b', text_lower)
    if not exp_match:
        return data  # Return data if no Experience section found

    exp_start = exp_match.start()

    # Initialize end of Experience section as end of text
    exp_end = len(text)

    # Find the closest next section header after 'experience'
    for sec in next_sections:
        sec_match = re.search(r'\b' + re.escape(sec) + r'\b', text_lower[exp_start + 1:])
        if sec_match:
            potential_end = exp_start + 1 + sec_match.start()
            if potential_end < exp_end:
                exp_end = potential_end

    # Extract and clean the Experience section
    experience_text = text[exp_start:exp_end].strip()
    experience_text = re.sub(r'\n{2,}', '\n', experience_text)
    data["Experience"] = experience_text # Assign the extracted experience text to data["Experience"]


    return data

Resume Text Extraction Check

In [45]:
import os

folder_path = "/content/drive/MyDrive/Colab Notebooks/"

# Loop through all files in folder
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf") or filename.endswith(".docx"):
        file_path = os.path.join(folder_path, filename)
        text = extract_text(file_path)
        info = extract_structured_data(text) # Pass the extracted text here

        print(f"Resume: {filename}")
        for k, v in info.items():
            print(f"{k}: {v}\n")
        print("="*80)

Resume: resume_juanjosecarin.pdf
Name: Hive

Email: juanjose.carin@gmail.com

Phone: 94041 650-336

Education: Education 2016 University of California, Berkeley Master of Information and Data Science GPA: 3.93 Relevant courses: • Machine Learning • Machine Learning at Scale • Storing and Retrieving Data • Field Experiments • Applied Regression and Time Series Analysis • Exploring and Analyzing Data • Data Visualization and Communication • Research Design and Applications for Data Analysis 2014 Universidad Politécnica de Madrid M.S. in Statistical and Computational Information Processing GPA: 3.69 Relevant courses: • Data Mining • Multivariate Analysis • Time Series • Neural Networks and Statistical Learning • Regression and Prediction Methods • Optimization Techniques • Monte Carlo Techniques • Numerical Methods in Finance • Stochastic Models in Finance • Bayesian Networks 2005 Universidad Politécnica de Madrid M.S. in Telecommunication Engineering GPA: 3.03 Focus Area: Radio communica

Matching Resume to Job Description

In [46]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [47]:
## Loading a Pretrained Embedding Model

model = SentenceTransformer('all-MiniLM-L6-v2')

In [48]:
## Input manual job description

job_description = """
We are looking for a Data Analyst skilled in Python, SQL, Excel, and Power BI.
The ideal candidate should have experience with data visualization and basic machine learning.
"""

In [49]:
## Computing Similarity b/w JD and Resume

def compute_similarity(resume_text, job_description):
    # Encode both texts into embeddings
    embeddings = model.encode([resume_text, job_description], convert_to_tensor=True)
    # Compute cosine similarity
    score = util.pytorch_cos_sim(embeddings[0], embeddings[1])
    return float(score)

In [50]:
## Apply to all uploaded resumes

resume_folder = "/content/drive/MyDrive/Colab Notebooks/"

results = []

# Loop through all PDF files in the folder
for filename in os.listdir(resume_folder):
    if filename.lower().endswith(".pdf"):   # only process PDFs
        filepath = os.path.join(resume_folder, filename)

        # Extract text from the resume
        text = extract_text(filepath)

        # Compute similarity with job description
        similarity_score = compute_similarity(text, job_description)

        # Append result
        results.append((filename, similarity_score))

# Sort by similarity score (descending order)
results = sorted(results, key=lambda x: x[1], reverse=True)

# Display the results
for name, score in results:
    print(f"{name} --> Match Score: {score:.4f}")

resume_juanjosecarin.pdf --> Match Score: 0.4401
Kallu Sujeev Reddy.pdf --> Match Score: 0.3438


Deploying to Streamlit

In [51]:
!pip install streamlit

In [52]:
%%writefile app.py
import streamlit as st
import fitz
import docx2txt
import re
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# --- Helper Functions ---
def extract_text(file_path):
    text = ""
    if file_path.name.endswith(".pdf"):
        with fitz.open(stream=file_path.read(), filetype="pdf") as pdf:
            for page in pdf:
                text += page.get_text("text")
    elif file_path.name.endswith(".docx"):
        text = docx2txt.process(file_path)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def compute_similarity(resume_text, job_description):
    embeddings = model.encode([resume_text, job_description], convert_to_tensor=True)
    score = util.pytorch_cos_sim(embeddings[0], embeddings[1])
    return float(score)

# --- Streamlit App ---
st.title("AI Resume Screener (Beginner Project)")
st.write("Upload resumes and a job description to rank candidates by relevance!")

# Job Description input
job_description = st.text_area("Paste Job Description Here:")

# Resume uploads
uploaded_files = st.file_uploader("Upload Resumes (PDF or DOCX)", type=["pdf", "docx"], accept_multiple_files=True)

if st.button("Analyze Resumes"):
    if not job_description or not uploaded_files:
        st.warning("Please upload resumes and provide a job description.")
    else:
        results = []
        for resume in uploaded_files:
            text = extract_text(resume)
            score = compute_similarity(text, job_description)
            results.append({"Filename": resume.name, "Match Score": round(score, 3)})

        df = pd.DataFrame(results).sort_values(by="Match Score", ascending=False)
        st.success("Analysis Complete! See below:")
        st.dataframe(df)


Writing app.py
